https://developers.google.com/gmail/api/quickstart/python

Todo: 
- Find only emails from a given recipient (FirstTutors)

In [10]:
import os.path
import base64
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [11]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

In [12]:
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('my_cred_file.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [13]:
service = build('gmail', 'v1', credentials=creds)
results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="from:enquiries@firsttutors.co.uk").execute()
messages = results.get('messages',[])

In [14]:
messages

[{'id': '18460676dae58b5c', 'threadId': '18460676dae58b5c'},
 {'id': '1845d054a8a6921c', 'threadId': '1845d054a8a6921c'},
 {'id': '1845cc135c2e4f57', 'threadId': '1845cc135c2e4f57'},
 {'id': '184531207a082275', 'threadId': '184531207a082275'},
 {'id': '184530e7286d1655', 'threadId': '184530e7286d1655'},
 {'id': '184361ff1d67f463', 'threadId': '184361ff1d67f463'},
 {'id': '184214f6dd0a9e66', 'threadId': '184214f6dd0a9e66'},
 {'id': '184214ea3fb21cb4', 'threadId': '184214ea3fb21cb4'},
 {'id': '1840dca8feab41a5', 'threadId': '1840dca8feab41a5'},
 {'id': '1840cb5158fcabd2', 'threadId': '1840cb5158fcabd2'},
 {'id': '18404fd54bd4449a', 'threadId': '18404fd54bd4449a'},
 {'id': '184012d694702f9b', 'threadId': '184011c621840019'},
 {'id': '18401220f4464094', 'threadId': '184011c621840019'},
 {'id': '184011c621840019', 'threadId': '184011c621840019'},
 {'id': '1840116d51b9654f', 'threadId': '1840116d51b9654f'},
 {'id': '183f931101be1525', 'threadId': '183f931101be1525'},
 {'id': '183f06ff42bb4cc

In [15]:
message = messages[0]

In [16]:
msg = service.users().messages().get(userId='me', id=message['id']).execute()  

In [17]:
df = pd.json_normalize(msg['payload']['parts'][0]['body'])
df

,size,data
0,1338,e1BSRVZJRVc6IFlvdXIgcG90ZW50aWFsIHN0dWRlbnQsIH...


In [18]:
body_message_encoded = msg['payload']['parts'][0]['body']['data']
byte_code = base64.urlsafe_b64decode(body_message_encoded)
body_message = byte_code.decode("utf-8")
body_message

"{PREVIEW: Your potential student, tlhalefang, may like to know if you would like to teach them University (or further higher)Maths on First Tutors}*** Is tlhalefang waiting for your decision? ***\r\n\r\nMake Decision ( https://www.firsttutors.com/uk/members/tutor-request-detail.php?tutee_request_id=1698291&al=GGWScgrryTA94NcVv3uxOHzKq%2F7VChCrtpuwzCy2xJ3vLUzIitDORSfhl2DKRuxH3o9yz58ddFKS9%2Fw1WDiz0w%3D%3D&utm_source=SendGrid&utm_medium=Email&utm_campaign=template_118 )tlhalefang has requested your help with University (or further higher)Maths. Are you ready to accept their request for tuition? \r\n\r\nPlease Note: We monitor response times / acceptance rates and use them to rank profiles in search results.  Please ensure that you respond promptly to all student requests. We would prefer you to decline than to leave a client waiting. \r\n\r\nIf you are not yet in a position to accept or decline this request please feel free to ignore this email reminder.\r\n\r\nIf you do not have availa